In [ ]:
import pandas as pd
import xgboost as xgb

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/datahack.av/AmExpert 2021/Preprocessed/train_data.csv')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/datahack.av/AmExpert 2021/Preprocessed/test_data.csv')

In [ ]:
train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,is_train,B1_P16,B1_P13,B1_P20,B1_P11,B1_P8,B1_P17,B1_P21,B1_P12,B1_P10,B1_P19,B1_P2,B1_P00,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18,current_holdings
0,CC264719,Male,41,14,0,C1,S3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,CC209679,Female,47,14,1,C1,S2,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,CC319633,Female,59,14,0,C2,S2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,CC231413,Female,32,16,0,C1,S2,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,CC259633,Male,30,15,0,C2,S3,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
test.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,is_train,B1_P16,B1_P13,B1_P20,B1_P11,B1_P8,B1_P17,B1_P21,B1_P12,B1_P10,B1_P19,B1_P2,B1_P00,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18,current_holdings
0,CC372708,Female,31,31,0,C2,S3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,CC216072,Male,28,37,1,C1,S2,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,CC387629,Male,31,12,0,C2,S3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,CC389228,Female,55,11,0,C2,S2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,CC394445,Male,51,49,1,C2,S1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
train.columns

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'is_train', 'B1_P16', 'B1_P13', 'B1_P20', 'B1_P11',
       'B1_P8', 'B1_P17', 'B1_P21', 'B1_P12', 'B1_P10', 'B1_P19', 'B1_P2',
       'B1_P00', 'B1_P18', 'B1_P15', 'B1_P6', 'B1_P9', 'B1_P7', 'B1_P3',
       'B1_P5', 'B1_P4', 'B1_P1', 'B1_P14', 'B2_P8', 'B2_P3', 'B2_P00',
       'B2_P6', 'B2_P12', 'B2_P16', 'B2_P1', 'B2_P9', 'B2_P10', 'B2_P13',
       'B2_P4', 'B2_P5', 'B2_P7', 'B2_P11', 'B2_P2', 'B2_P15', 'B2_P17',
       'B2_P14', 'B2_P20', 'B2_P18', 'current_holdings'],
      dtype='object')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37748 entries, 0 to 37747
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Customer_ID        37748 non-null  object
 1   Gender             37748 non-null  object
 2   Age                37748 non-null  int64 
 3   Vintage            37748 non-null  int64 
 4   Is_Active          37748 non-null  int64 
 5   City_Category      37748 non-null  object
 6   Customer_Category  37748 non-null  object
 7   is_train           37748 non-null  int64 
 8   B1_P16             37748 non-null  int64 
 9   B1_P13             37748 non-null  int64 
 10  B1_P20             37748 non-null  int64 
 11  B1_P11             37748 non-null  int64 
 12  B1_P8              37748 non-null  int64 
 13  B1_P17             37748 non-null  int64 
 14  B1_P21             37748 non-null  int64 
 15  B1_P12             37748 non-null  int64 
 16  B1_P10             37748 non-null  int64

In [ ]:
cat_cols = ['Gender', 'Is_Active', 'City_Category','Customer_Category'] #,
cols_to_drop = ['Customer_ID','is_train']

In [ ]:
B1_cols = [col for col in list(train.columns) if col.startswith('B1')]
B2_cols = [col for col in list(train.columns) if col.startswith('B2')]

In [ ]:
cat_cols = B1_cols + cat_cols
target_cols = B2_cols

In [ ]:
train = train.drop(columns = cols_to_drop)

In [ ]:
test = test.drop(columns = cols_to_drop)

In [ ]:
# df_train = pd.get_dummies(train, columns=cat_cols_i)

# df_test = pd.get_dummies(test, columns=cat_cols_i)

In [ ]:
X_train = train.drop(columns = target_cols)
y_train = train[target_cols]

In [ ]:
X_train

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P16,B1_P13,B1_P20,B1_P11,B1_P8,B1_P17,B1_P21,B1_P12,B1_P10,B1_P19,B1_P2,B1_P00,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,current_holdings
0,Male,41,14,0,C1,S3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Female,47,14,1,C1,S2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,Female,59,14,0,C2,S2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Female,32,16,0,C1,S2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,Male,30,15,0,C2,S3,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37743,Male,28,24,1,C1,S2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
37744,Male,36,11,0,C2,S2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
37745,Male,44,26,1,C2,S2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
37746,Female,56,15,1,C1,S2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [ ]:
y_train

,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37743,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
37744,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
37745,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
37746,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for cols in y_train.columns:
    print(cols)
    print(y_train[cols].unique())

B2_P8
[1 0]
B2_P3
[0 1]
B2_P00
[0 1]
B2_P6
[0 1]
B2_P12
[0 1]
B2_P16
[0 1]
B2_P1
[0 1]
B2_P9
[0 1]
B2_P10
[0 1]
B2_P13
[0 1]
B2_P4
[0 1]
B2_P5
[0 1]
B2_P7
[0 1]
B2_P11
[0 1]
B2_P2
[0 1]
B2_P15
[0 1]
B2_P17
[0 1]
B2_P14
[0 1]
B2_P20
[0 1]
B2_P18
[0 1]


In [ ]:
cat_cols2 = ['Gender','Customer_Category']


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
X_train['Gender']=le.fit_transform(X_train['Gender'])

In [ ]:
X_train.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P16,B1_P13,B1_P20,B1_P11,B1_P8,B1_P17,B1_P21,B1_P12,B1_P10,B1_P19,B1_P2,B1_P00,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,current_holdings
0,1,41,14,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,47,14,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,59,14,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,32,16,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,1,30,15,0,1,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
X_train['City_Category']=le.fit_transform(X_train['City_Category'])

In [ ]:
X_train['Customer_Category']=le.fit_transform(X_train['Customer_Category'])

In [ ]:
X_train['City_Category'] = X_train['City_Category'].astype('category')
X_train['Customer_Category'] = X_train['Customer_Category'].astype('category')
cat_cols2 = ['Gender','City_Category','Customer_Category']
X_train[cat_cols2] = X_train[cat_cols2].apply(lambda x: x.cat.codes)

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 15 kB/s 


In [ ]:
# # Get all object variables from X_train
# cat_vars = [var for var in X_train.columns if X_train[var].dtype == "O"]

In [ ]:
# cat_vars

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
import catboost
classifier = MultiOutputClassifier(catboost.CatBoostClassifier(cat_features=cat_cols,verbose=1))

In [ ]:
classifier.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
4:	learn: 0.2019165	total: 174ms	remaining: 34.6s
5:	learn: 0.1599611	total: 214ms	remaining: 35.4s
6:	learn: 0.1278228	total: 242ms	remaining: 34.3s
7:	learn: 0.1029647	total: 267ms	remaining: 33.1s
8:	learn: 0.0839292	total: 308ms	remaining: 33.9s
9:	learn: 0.0680350	total: 344ms	remaining: 34s
10:	learn: 0.0565579	total: 378ms	remaining: 34s
11:	learn: 0.0475130	total: 425ms	remaining: 35s
12:	learn: 0.0403548	total: 458ms	remaining: 34.8s
13:	learn: 0.0343603	total: 492ms	remaining: 34.6s
14:	learn: 0.0298848	total: 526ms	remaining: 34.5s
15:	learn: 0.0254473	total: 558ms	remaining: 34.3s
16:	learn: 0.0223984	total: 593ms	remaining: 34.3s
17:	learn: 0.0197048	total: 630ms	remaining: 34.4s
18:	learn: 0.0179365	total: 671ms	remaining: 34.6s
19:	learn: 0.0164379	total: 704ms	remaining: 34.5s
20:	learn: 0.0147298	total: 739ms	remaining: 34.5s
21:	learn: 0.0136004	total: 778ms	remaining: 34.6s
22:	learn: 0.0128193	total: 812ms	remaining

MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7f232ac9e590>)

In [ ]:
test.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,B1_P16,B1_P13,B1_P20,B1_P11,B1_P8,B1_P17,B1_P21,B1_P12,B1_P10,B1_P19,B1_P2,B1_P00,B1_P18,B1_P15,B1_P6,B1_P9,B1_P7,B1_P3,B1_P5,B1_P4,B1_P1,B1_P14,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18,current_holdings
0,Female,31,31,0,C2,S3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,Male,28,37,1,C1,S2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,Male,31,12,0,C2,S3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Female,55,11,0,C2,S2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,Male,51,49,1,C2,S1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
test['City_Category']=le.fit_transform(test['City_Category'])
test['Customer_Category']=le.fit_transform(test['Customer_Category'])
test['Gender']=le.fit_transform(test['Gender'])

In [ ]:
preds = classifier.predict_proba(test)

In [ ]:
preds_cal = []
for item in preds:
    preds_cal.append( [local_item[1] for local_item in item] )

In [ ]:
import numpy as np
df_catboost = pd.DataFrame(np.array(preds_cal).T, columns=B2_cols)

In [ ]:
df_catboost

,B2_P8,B2_P3,B2_P00,B2_P6,B2_P12,B2_P16,B2_P1,B2_P9,B2_P10,B2_P13,B2_P4,B2_P5,B2_P7,B2_P11,B2_P2,B2_P15,B2_P17,B2_P14,B2_P20,B2_P18
0,0.581978,0.087672,0.000001,0.039131,0.000193,3.641035e-07,0.053917,0.069865,0.241838,2.004370e-06,0.191334,0.044325,0.032937,1.470207e-06,5.763499e-07,2.005294e-06,1.333070e-07,5.380649e-07,1.183438e-06,9.655665e-08
1,0.535470,0.125416,0.000002,0.026989,0.000382,5.912117e-07,0.326817,0.034772,0.260164,1.058201e-05,0.104698,0.019860,0.025908,3.276984e-07,1.133892e-06,1.631900e-07,3.993436e-08,3.755051e-07,1.276624e-06,1.500517e-07
2,0.011997,0.002489,0.108385,0.011090,0.002839,6.964192e-01,0.001916,0.002840,0.003892,3.602058e-01,0.004422,0.001588,0.003981,1.218410e-03,2.448420e-04,1.139093e-05,2.845683e-04,4.376448e-08,5.403417e-07,1.382426e-07
3,0.560395,0.054587,0.000007,0.039455,0.025784,4.263137e-05,0.044558,0.081413,0.225757,1.164002e-04,0.054555,0.019759,0.029547,1.258228e-05,6.527884e-06,2.045802e-07,2.243072e-06,4.859163e-07,6.890985e-06,2.310450e-07
4,0.153306,0.034802,0.523455,0.017935,0.046556,6.508878e-06,0.112416,0.006269,0.004142,9.227836e-05,0.018324,0.001661,0.022994,2.808680e-06,2.303267e-07,9.550209e-08,8.153456e-08,2.032667e-06,3.429036e-06,1.053075e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20322,0.249892,0.003868,0.222952,0.170045,0.026154,7.242649e-05,0.073946,0.001093,0.062394,1.962416e-04,0.029974,0.030695,0.040951,3.970434e-07,6.970409e-07,1.397990e-03,3.344676e-07,1.796413e-06,7.211716e-07,3.737180e-08
20323,0.594218,0.003348,0.042077,0.112113,0.206851,5.839981e-05,0.002673,0.003082,0.025382,2.863204e-05,0.019549,0.010717,0.034979,2.587127e-05,8.723577e-06,7.763959e-04,2.952067e-08,4.987074e-06,3.267673e-07,4.065653e-08
20324,0.208802,0.029106,0.181531,0.142926,0.025278,9.971584e-05,0.022838,0.150907,0.025187,1.942346e-05,0.027098,0.012275,0.028190,1.050603e-05,2.631697e-07,3.307366e-07,1.663028e-07,5.032432e-08,4.527502e-07,2.752917e-08
20325,0.002804,0.229183,0.000005,0.641797,0.000026,1.129407e-07,0.924239,0.000080,0.000053,3.889606e-07,0.065311,0.044105,0.364233,2.243589e-05,1.594035e-04,3.746131e-05,3.922649e-07,2.872062e-05,6.236426e-06,1.337545e-07


In [ ]:
df_catboost.to_csv('/content/drive/MyDrive/datahack.av/AmExpert 2021/results/catboost_preds.csv', index=False)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer(sparse_output=True)

# train = train.join(
#             pd.DataFrame.sparse.from_spmatrix(
#                 mlb.fit_transform(train.pop('Product_Holding_B1')),
#                 index=train.index,
#                 columns=mlb.classes_))

In [ ]:
# df_melt = train.assign(names=train.Product_Holding_B1.str.split(","))
  


In [ ]:
df2 = pd.DataFrame(train['Product_Holding_B1'].str.split().values.tolist())